In [1]:
import os
import sys
import math
import GPUtil
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
from tqdm import tqdm_notebook as tqdm, tnrange
from transformers.tokenization_bert import BertTokenizer
from transformers import BertModel
from transformers.optimization import AdamW, get_linear_schedule_with_warmup

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

I1204 03:38:25.988224 139745022572352 file_utils.py:39] PyTorch version 1.1.0 available.


'/work'

In [2]:
sys.path.append(w_dir+'/fgc_support_retri')

In [3]:
import config
from utils import read_fgc
from fgc_preprocess import SerContextDataset, BertSpanIdx

I1204 03:38:26.424961 139745022572352 corenlp.py:42] Using an existing server http://140.109.19.191:9000
I1204 03:38:27.428436 139745022572352 corenlp.py:118] The server is available.


In [4]:
items = read_fgc(config.FGC_DEV, eval=True)

no gold supporting evidence
{'QID': 'D032Q10', 'QTYPE': '进阶题', 'QTEXT': '第二次簽訂的北美貿易協定從簽署至生效過了幾日?', 'SENTS': [{'text': '第二次签订的北美贸易协定从签署至生效过了几日?', 'start': 0, 'end': 23}], 'ANSWER': [{'ATEXT': '資訊不足無法判定', 'ATOKEN': [{'text': '资讯不足无法判定', 'start': -1}], 'ATEXT_CN': '资讯不足无法判定'}], 'ATYPE': 'Date-Duration', 'AMODE': 'Date-Duration', 'ASPAN': [], 'SHINT': [], 'QTEXT_CN': '第二次签订的北美贸易协定从签署至生效过了几日?'}
no gold supporting evidence
{'QID': 'D049Q04', 'QTYPE': '申论', 'QTEXT': '「雅婷逐字稿」的命名起源為何?', 'SENTS': [{'text': '「雅婷逐字稿」的命名起源为何?', 'start': 0, 'end': 15}], 'ANSWER': [{'ATEXT': '', 'ATOKEN': [{'text': '', 'start': 0}], 'ATEXT_CN': ''}], 'ATYPE': 'Event', 'AMODE': 'Single-Span-Extraction', 'ASPAN': [], 'SHINT': [], 'QTEXT_CN': '「雅婷逐字稿」的命名起源为何?'}
no gold supporting evidence
{'QID': 'D086Q03', 'QTYPE': '申论', 'QTEXT': '不可再生能源的意義是什麼？', 'SENTS': [{'text': '不可再生能源的意义是什么？', 'start': 0, 'end': 13}], 'ANSWER': [{'ATEXT': '', 'ATOKEN': [{'text': '', 'start': 0}], 'ATEXT_CN': ''}], 'ATYPE': 'Object', 'AMODE': 'Sing

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
dev_set = SerContextDataset(items, transform=torchvision.transforms.Compose([BertSpanIdx(tokenizer)]))

I1204 03:38:30.049935 139745022572352 tokenization_utils.py:375] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00


In [6]:
dev_set[1]

{'QID': 'D004Q02',
 'SENTS': [{'text': '元祐元年（1086年），', 'start': 0, 'end': 12},
  {'text': '宋哲宗即位，', 'start': 12, 'end': 18},
  {'text': '高太皇太后垂帘听政，', 'start': 18, 'end': 28},
  {'text': '回朝任礼部郎中、中书舍人、翰林学士，', 'start': 28, 'end': 46},
  {'text': '元祐四年（1089年）拜龙图阁学士，', 'start': 46, 'end': 64},
  {'text': '曾出任杭州、颍州等知州职务，', 'start': 64, 'end': 78},
  {'text': '官至礼部尚书。', 'start': 78, 'end': 85},
  {'text': '\n绍圣元年（1094年）被哲宗贬谪至惠州、儋州（海南岛）。', 'start': 85, 'end': 114},
  {'text': '\n元符三年（1100年），', 'start': 114, 'end': 127},
  {'text': '宋徽宗即位，', 'start': 127, 'end': 133},
  {'text': '向太后垂帘听政，', 'start': 133, 'end': 141},
  {'text': '下诏让苏轼北还。', 'start': 141, 'end': 149},
  {'text': '\n建中靖国元年（1101年），', 'start': 149, 'end': 164},
  {'text': '夏天因冷饮过度，', 'start': 164, 'end': 172},
  {'text': '下痢不止，又误服黄芪，', 'start': 172, 'end': 183},
  {'text': '结果病情恶化，', 'start': 183, 'end': 190},
  {'text': '「齿间出血如蚯蚓者无数」，', 'start': 190, 'end': 203},
  {'text': '七月二十八日于常州孙氏馆病卒，', 'start': 203, 'end': 218},
  {'text': 